## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [3]:
df = pd.read_csv('../../../../../../jaeyeun/01_nh_poc/17_add_prep_text_to_excel/split_70_15_15_prep_v9.csv')

df.head()

,file_name,label,raw_text,dataset_n,train_val_test,prep_v9_text
0,R1509261.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 9. 4...,0,0,온라인N 제도N 혁신N 관호N 미래N 사이트N 플러그인N 지원N 대응N 현황N 공개...
1,R2003733.txt,0,보도일시 2020. 3. 18.(수) 조간(온라인 3. 17. 12:00)부터 보도...,0,0,온라인N 정보N 산업N 기반N 유승N ictS 분야N 창업N 벤처N 지원N 사업N ...
2,D1507076-1.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 7. 1...,0,0,이후N 연구N 예산N 내년도N 연구N 개발N r&dN 경제N 혁신N 미래N 성장N ...
3,R2005031.txt,0,보 도 자 료\n배포일시 2020. 4. 29.(수) 총 4매(본문2) 담당 부서 ...,0,0,첨단N 항공N 과장N 문석N 홍일산N 이후N 가능N 일상N 시대N 개막N 시행N 전...
4,R2006226.txt,0,<전매체> 2020년 6월 3일(수) 10:00(국무회의 개최시)부터 보도하여 주시...,0,0,국무회의N 개최N 문의N 기획N 재정관N 과장N 서기관N 위기N 혁신N 기회N 벤처...


* Train : KDI 70+15 (train+val)
* Val : KDI 15(test)
* Test : legal 273(train+val+test)

In [5]:
X_train_df = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]
X_val_df = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]
X_test_df = df[df['dataset_n'] != 0]
y_train = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]['label']
y_val = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]['label']
y_test = df[df['dataset_n'] != 0]['label']

In [6]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(9174, 6)
(1626, 6)
(273, 6)
(9174,)
(1626,)
(273,)


In [7]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

In [8]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v9_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [9]:
X_train = X_train_tfidf.transform(X_train_df['prep_v9_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v9_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v9_text'])

In [10]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(9174, 100000)
(1626, 100000)
(273, 100000)


## Classify

* XGBoost RandomSearch

In [11]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

In [12]:
xgb_clf = xgb.XGBClassifier()

xgb_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "merror",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
xgb_random = RandomizedSearchCV(estimator = xgb_clf,
                                param_distributions = xgb_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
xgb_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 51.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 85.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 128.9min
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed: 174.7min remaining: 189.3min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed: 197.1min remaining: 137.0min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed: 213.1min remaining: 91.3min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed: 222.1min remaining: 52.1min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 236.9min remaining: 20.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 255.6min finished


[0]	validation_0-merror:0.20722	validation_1-merror:0.24908
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 20 rounds.
[1]	validation_0-merror:0.17964	validation_1-merror:0.22263
[2]	validation_0-merror:0.16132	validation_1-merror:0.21710
[3]	validation_0-merror:0.15184	validation_1-merror:0.21279
[4]	validation_0-merror:0.14290	validation_1-merror:0.20603
[5]	validation_0-merror:0.13484	validation_1-merror:0.20910
[6]	validation_0-merror:0.12971	validation_1-merror:0.21033
[7]	validation_0-merror:0.12339	validation_1-merror:0.21341
[8]	validation_0-merror:0.11827	validation_1-merror:0.20972
[9]	validation_0-merror:0.11522	validation_1-merror:0.20726
[10]	validation_0-merror:0.10955	validation_1-merror:0.20172
[11]	validation_0-merror:0.10541	validation_1-merror:0.19619
[12]	validation_0-merror:0.10039	validation_1-merror:0.19496
[13]	validation_0-merror:0.09865	validation_1-merror:0

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [13]:
pred_train = xgb_random.best_estimator_.predict(X_train)
pred_val = xgb_random.best_estimator_.predict(X_val)
pred_test = xgb_random.best_estimator_.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.9965118814039677
0.8382533825338253
0.38095238095238093


In [16]:
xgb_random.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=17,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
xgb_cv_result_df = pd.DataFrame(xgb_random.cv_results_)

In [18]:
df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([xgb_cv_result_df.loc[i, "params"]]))
    
xgb_param_table = pd.concat(df_list)

In [19]:
xgb_param_table['mean_test_score'] = xgb_cv_result_df['mean_test_score'].values

In [20]:
xgb_param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,mean_test_score
0,0.6,7,25,0.010,0.786353
0,1.0,7,12,0.015,0.789080
0,0.7,3,7,0.015,0.791912
0,1.0,3,9,0.015,0.794202
0,0.7,1,7,0.010,0.796055
0,0.7,3,12,0.010,0.796818
0,0.9,3,12,0.010,0.797907
0,0.6,7,15,0.025,0.799761
0,0.9,1,25,0.025,0.803357
0,0.8,5,9,0.015,0.803358
